<a href="https://colab.research.google.com/github/cagBRT/PerformanceEnhancement/blob/main/pandas_performance_enhancement_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
np.random.seed(3)
random_numbers_1=np.random.randint(10e1, size=100000000)
random_numbers_2=np.random.randint(10e2, size=100000000)
random_numbers_3=np.random.randint(10e3, size=100000000)

df = pd.DataFrame({
    'random_numbers_1':random_numbers_1,
    'random_numbers_2':random_numbers_2,
    'random_numbers_3':random_numbers_3,
})
df

DataFrame Iterrows:
**Iterrows() allows you to iterate through a pandas DataFrame row by row and it’s usually an approach to be avoided.** As in this case, we couldn’t even finish the code within the time limit we set.

In [ ]:
%%time
def standard_scalar_iterrows(pandas_df:pd.DataFrame,
                             mean_ps:pd.Series,
                             std_series:pd.Series,
                             )->pd.DataFrame:
  """Iterate through the rows of the Pandas DataFrame and do the standard scaler calculation row by row"""

  for index, row in pandas_df.iterrows():
    pandas_df['scaled_random_numbers_2']=\
    (row[r'random_numbers_2']-mean_ps)/std_series
  return pandas_df


The following code will take a very long time to run, **don't use iterrows for large datasets**

In [ ]:
mean_ps=np.mean(df['random_numbers_2'])
std_ps=np.std(df['random_numbers_2'])

data_df=standard_scalar_iterrows(df,
                                 mean_ps,
                                 std_ps)